In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lvmdrp.scripts import preproc


OBS_NAME = "dummy-lvm-obs.fits.gz"

import re


type2fmt = {
    "bool": "L",
    "binary": "L",
    "logical": "L",
    "byte": "X",
    "char": "A",
    "int16": "I",
    "int32": "J",
    "int64": "K",
    "float32": "E",
    "float64": "D"
}
type2dmy = dict(zip(type2fmt.keys(),[True, True, True, 0, "x", -9999, -9999, -9999, -9999.9, -9999.9]))

pattern = re.compile(r"\[(\d+)\]")

In [2]:
import pandas as pd


raw_data_url = "_dummy_data/Header_ metadata model from the LVM lab data - LVM - SDSS.html"

desimodel_raw_tables = []
for table in pd.read_html(raw_data_url):
    clean_table = table.fillna(-9999)
    clean_table = clean_table.applymap(lambda s: s.replace("’","").replace("‘","").replace("…","") if type(s)==str else s)
    clean_table = clean_table.applymap(lambda s: pd.to_numeric(s, errors="ignore", downcast="integer"))
    desimodel_raw_tables.append(clean_table)

clean_table.columns = clean_table.loc[0]
clean_table = clean_table.drop(labels=0).rename(columns={"Value(Examples)":"Example Value", "Information":"Comment", "Comment":"Type"})

In [3]:
# generate dummy observations

import itertools as it

dummy = fits.HDUList()

hdu = fits.PrimaryHDU(header=fits.Header(cards=map(tuple, clean_table.drop(columns="Type").values.tolist())))

dummy.append(hdu)
dummy.writeto(os.path.join("_dummy_data/",OBS_NAME), overwrite=True)
dummy.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      39   ()      
